<a href="https://colab.research.google.com/github/mmsamiei/final-ms-thesis-code/blob/master/phase1/hello_faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nlp
!pip install transformers
!pip install faiss-cpu

     |████████████████████████████████| 1.7MB 7.7MB/s 
     |████████████████████████████████| 17.2MB 240kB/s 
     |████████████████████████████████| 245kB 46.8MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
     |████████████████████████████████| 778kB 8.2MB/s 
     |████████████████████████████████| 890kB 23.5MB/s 
     |████████████████████████████████| 3.0MB 36.9MB/s 
     |████████████████████████████████| 1.1MB 45.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=846581663c05c7d98e957cddc0688e064b0bc3e556b2dfd62f5cd767ca67aa36
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 7.2MB 7.4MB/s 


In [13]:
from transformers import AutoTokenizer, AutoModel
import torch
torch.set_grad_enabled(False)
ctx_encoder = AutoModel.from_pretrained("google/bert_uncased_L-2_H-128_A-2")
ctx_tokenizer = AutoTokenizer.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

In [30]:
from nlp import load_dataset
ds = load_dataset('crime_and_punish', split='train[:500]')
ds_with_embeddings = ds.map(lambda example: {'embeddings': ctx_encoder(**ctx_tokenizer(example["line"], return_tensors="pt"))[0][0][0].numpy()})

In [31]:
ds_with_embeddings.add_faiss_index(column='embeddings')

Dataset(features: {'embeddings': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), 'line': Value(dtype='string', id=None)}, num_rows: 500)

In [32]:
q_encoder = AutoModel.from_pretrained("google/bert_uncased_L-2_H-128_A-2")
q_tokenizer = AutoTokenizer.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

In [37]:
question = "sex"
question_embedding = q_encoder(**q_tokenizer(question, return_tensors="pt"))[0][0][0].numpy()
scores, retrieved_examples = ds_with_embeddings.get_nearest_examples('embeddings', question_embedding, k=10)

In [38]:
retrieved_examples["line"]

['nails.\r\n',
 '\r\n',
 '\r\n',
 '\r\n',
 '\r\n',
 '\r\n',
 '\r\n',
 '\r\n',
 '\r\n',
 '\r\n']

In [39]:
ds_with_embeddings.save_faiss_index('embeddings', 'my_index.faiss')

In [41]:
ds = load_dataset('crime_and_punish', split='train[:500]')
ds.load_faiss_index('embeddings', 'my_index.faiss')